In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


In [4]:
def processData(path):
    def calculateAge(month: str, lease_commence_date: float) -> int:
        # calculate years of lease remaining, using 99 years as default, round up to the nearest year
        sold_year = month.strftime('%Y')
        return max(int(int(sold_year) - lease_commence_date ),0)

    def classifyStorey(storey_range):
        # floor = average of start and end storey
        start = int(storey_range[:2])
        end = int(storey_range[-2:])
        return (start + end)//2

    def classifyModel(model):
        if 'Maisonette' in model or 'Generation' in model:
            return 'Maisonette'
        elif 'Type' in model:
            return 'Type'
        elif 'Model' in model:
            return 'Model'
        else: return model

    df = pd.read_csv(path)
    df['month'] = pd.to_datetime(df['month'])
    df['age'] = df.apply(
        lambda x: calculateAge(x['month'], x['lease_commence_date']), axis=1)
    df['general_model'] = df['flat_model'].apply(classifyModel)
    df['storey'] = df['storey_range'].apply(classifyStorey)

    df['sold_year'] = df['month'].dt.strftime('%Y')
    df=df.dropna(axis=1)
    return df.copy()


In [5]:
train_df = processData('./train.csv')
train_df.head()

,town,month,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,age,general_model,storey,sold_year
0,GEYLANG,2006-01-01,4 ROOM,328,UBI AVE 1,04 TO 06,84.0,Simplified,1985.0,175000.0,21,Simplified,5,2006
1,JURONG EAST,2003-02-01,5 ROOM,271,TOH GUAN RD,10 TO 12,120.0,Improved,1999.0,353000.0,4,Improved,11,2003
2,TOA PAYOH,2005-09-01,3 ROOM,205,TOA PAYOH NTH,10 TO 12,65.0,Improved,1973.0,215000.0,32,Improved,11,2005
3,PASIR RIS,2001-03-01,EXECUTIVE,508,PASIR RIS ST 52,04 TO 06,146.0,Apartment,1993.0,482000.0,8,Apartment,5,2001
4,JURONG WEST,2014-07-01,EXECUTIVE,656B,JURONG WEST ST 61,10 TO 12,133.0,Apartment,2001.0,613000.0,13,Apartment,11,2014


In [6]:
train_df.groupby('flat_model').mean().sort_values('resale_price')

,floor_area_sqm,lease_commence_date,resale_price,age,storey
flat_model,,,,,
2-room,52.217391,2001.130435,2.208696e+05,10.739130,3.173913
Simplified,79.045040,1986.915731,2.578829e+05,21.448952,6.065735
New Generation,77.714583,1981.350965,2.581721e+05,27.074838,6.629465
Standard,75.789245,1972.533620,2.874509e+05,35.930370,7.864882
Model A2,89.886304,1999.162198,2.884823e+05,9.814800,7.650368
Model A,97.451085,1994.385445,3.400544e+05,15.264277,7.860276
Improved,100.119078,1988.290615,3.582220e+05,21.072650,8.527715
Premium Apartment,106.603112,2003.800431,4.275922e+05,8.930915,9.291932
Adjoined flat,140.231931,1977.622438,4.843490e+05,31.411003,7.742179


In [7]:
features = ['town', 'general_model', 'flat_type',
            'storey', 'sold_year', 'age']


In [5]:
# independent variables
X = train_df[features]

# dependent variables
Y = train_df['resale_price']/train_df['floor_area_sqm']


In [6]:
# convert to dummy variable for category variable
X = pd.get_dummies(data=X,drop_first=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state=42)


In [7]:
# fit with linear regression
est= sm.OLS(Y, sm.add_constant(X)).fit()
print(est.summary())


                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.857
Method:                 Least Squares   F-statistic:                 4.142e+04
Date:                Tue, 22 Feb 2022   Prob (F-statistic):               0.00
Time:                        10:11:09   Log-Likelihood:            -3.5312e+06
No. Observations:              462515   AIC:                         7.063e+06
Df Residuals:                  462447   BIC:                         7.063e+06
Df Model:                          67                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


Since the F-statistic value is very large and Prob (F-statistic) is less than 0.05, we have sufficient evidence that there is a good amount of linear relationship between the target variable and the feature variables

In [8]:
test_df = processData('./test.csv')
test_df.head()

,town,month,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,age,general_model,storey,sold_year
0,YISHUN,2003-02-01,5 ROOM,167,YISHUN RING RD,04 TO 06,121.0,Improved,1987,16,Improved,5,2003
1,PUNGGOL,2019-04-01,4 ROOM,271A,PUNGGOL WALK,10 TO 12,93.0,Premium Apartment,2014,5,Premium Apartment,11,2019
2,ANG MO KIO,2013-11-01,3 ROOM,644,ANG MO KIO AVE 4,04 TO 06,75.0,New Generation,1980,33,Maisonette,5,2013
3,SERANGOON,2014-11-01,5 ROOM,258,SERANGOON CTRL DR,10 TO 12,115.0,Improved,1999,15,Improved,11,2014
4,YISHUN,2005-10-01,3 ROOM,216,YISHUN ST 21,07 TO 09,68.0,New Generation,1985,20,Maisonette,8,2005


In [9]:
X_test = test_df[features]

X_test=pd.get_dummies(X_test,drop_first=True)


In [10]:
y_pred = est.predict(sm.add_constant(X_test))

In [11]:
submission = test_df['floor_area_sqm']*y_pred

In [12]:
submission = pd.DataFrame(
    submission, columns=['Resale_price'])

submission['Index'] = np.arange(1, len(submission)+1)

submission = submission[['Index', 'Resale_price']]


In [13]:
submission.to_csv('submission.csv', index=False)


In [14]:
submission

,Index,Resale_price
0,1,225266.551785
1,2,456366.714204
2,3,379877.305794
3,4,620456.224515
4,5,145289.567812
...,...,...
115624,115625,182074.903597
115625,115626,183922.929992
115626,115627,426028.643429
115627,115628,171881.540851
